In [6]:
 
import pandas as pd
import numpy as np
from pymongo import MongoClient


# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["stepup"]
collection = db["shoes"]

# Fetch data from MongoDB
cursor = collection.find({}, {"_id": 0, "name": 1, "category": 1, "price": 1, "popularity": 1, "durability": 1})

# Create a DataFrame from MongoDB cursor
df = pd.DataFrame(list(cursor))

In [16]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd   
import numpy as np 

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#load dataset
df = pd.read_csv('C:\Capstone_OnlineStore\python\shoe_sales.csv')
df = pd.read_json('C:\Capstone_OnlineStore\mongodb\shoes.json')

# Check the columns of your DataFrame
print(df.columns)

<>:9: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\C'
<>:9: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\C'
C:\Users\wasadmin\AppData\Local\Temp\ipykernel_11412\2108910884.py:9: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv('C:\Capstone_OnlineStore\python\shoe_sales.csv')
C:\Users\wasadmin\AppData\Local\Temp\ipykernel_11412\2108910884.py:11: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_json('C:\Capstone_OnlineStore\mongodb\shoes.json')


Index(['addedTimestamp', 'id', 'brand', 'type', 'color', 'size', 'price'], dtype='object')


In [18]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
# Sample data
data = {
    'product_id': [1, 2, 3, 4, 5],
    'affordability': [5, 3, 4, 2, 1],
    'durability': [3, 4, 2, 5, 1],
    'popularity': [4, 5, 3, 2, 1]
}

df = pd.DataFrame(data)

In [24]:

# Features
X = df[['affordability', 'durability', 'popularity']]
print(X)

   affordability  durability  popularity
0              5           3           4
1              3           4           5
2              4           2           3
3              2           5           2
4              1           1           1


In [28]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming df is your original DataFrame read from JSON
df = pd.read_json("C:\Capstone_OnlineStore\mongodb\shoes.json")

# Select relevant columns
df_selected = df[['brand', 'type', 'color', 'size', 'price']].copy()


# Apply one-hot encoding to categorical variables
df_encoded = pd.get_dummies(df_selected, columns=['brand', 'type', 'color', 'size', 'price'])

knn = NearestNeighbors(n_neighbors=3).fit(df_encoded)

<>:5: SyntaxWarning: invalid escape sequence '\C'
<>:5: SyntaxWarning: invalid escape sequence '\C'
C:\Users\wasadmin\AppData\Local\Temp\ipykernel_11412\4010674713.py:5: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_json("C:\Capstone_OnlineStore\mongodb\shoes.json")


In [ ]:
selected_product_features = df_encoded[df_encoded['productID'] == 1]
selected_product_index = selected_product_features.index[0]
distances, indices = knn.kneighbors(selected_product_features)
recommended_product_indices = indices.flatten()
recommended_products = df_selected.iloc[recommended_product_indices]

print("Selected Product:")
print(df_selected.iloc[selected_product_index])
print("\nRecommended Products:")
print(recommended_products)